# Защита персональных данных клиентов

**Дано:** датасет с признаками: пол, возраст и зарплата застрахованного, количество членов его семьи. Целевой признак: количество страховых выплат клиенту за последние 5 лет.

**Задача:** защитить данные клиентов страховой компании «Хоть потоп». Разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию, и при преобразовании качество моделей машинного обучения не ухудшилось. 

**План:**
1. Загрузка, обзор данных. 
2. Проверка изменения качества линейной регрессии
3. Алгоритм преобразования данных для решения задачи 

## 1. Загрузка, обзор, предобработка данных

In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

from sklearn.metrics import r2_score

from sklearn.linear_model import LinearRegression

In [2]:
try:
    clients = pd.read_csv('/Users/galina//Desktop/учёба/спринт 10/insurance.csv') 
except:
    clients = pd.read_csv('/datasets/insurance.csv')

In [3]:
display(clients.head(5))
print('-----------------------------------------------------------------')
display(clients.tail(5))
print('-----------------------------------------------------------------')
display(clients.info())
print('-----------------------------------------------------------------')
display(clients.describe())
print('-----------------------------------------------------------------')
print('Кол-во пропусков:')
display(clients.isna().sum())
print('-----------------------------------------------------------------')
print('Кол-во дубликатов -', clients.duplicated().sum())

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


-----------------------------------------------------------------


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0
4999,1,28.0,40600.0,1,0


-----------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


None

-----------------------------------------------------------------


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


-----------------------------------------------------------------
Кол-во пропусков:


Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

-----------------------------------------------------------------
Кол-во дубликатов - 153


In [4]:
#смотрю распределение по уникальным значениям в столбцах:
for column in clients:
    print(clients[column].value_counts(normalize=True))
    print('-------------------------------------------')

0    0.501
1    0.499
Name: Пол, dtype: float64
-------------------------------------------
19.0    0.0446
25.0    0.0428
31.0    0.0424
26.0    0.0422
22.0    0.0418
27.0    0.0418
32.0    0.0412
28.0    0.0408
29.0    0.0406
30.0    0.0404
23.0    0.0404
21.0    0.0400
20.0    0.0390
36.0    0.0386
33.0    0.0382
24.0    0.0364
35.0    0.0358
34.0    0.0354
37.0    0.0294
39.0    0.0282
38.0    0.0278
41.0    0.0258
18.0    0.0234
40.0    0.0228
42.0    0.0186
43.0    0.0154
44.0    0.0148
45.0    0.0146
46.0    0.0120
47.0    0.0094
49.0    0.0074
50.0    0.0054
48.0    0.0052
52.0    0.0044
51.0    0.0042
53.0    0.0022
55.0    0.0018
54.0    0.0014
56.0    0.0010
59.0    0.0006
57.0    0.0004
58.0    0.0004
60.0    0.0004
61.0    0.0002
65.0    0.0002
62.0    0.0002
Name: Возраст, dtype: float64
-------------------------------------------
45800.0    0.0058
37100.0    0.0056
41500.0    0.0054
43200.0    0.0054
46800.0    0.0052
            ...  
17700.0    0.0002
70600.0    0.0002


In [5]:
#заменяю тип данных в столбцах Возраст и Зарплата на int:
clients['Возраст'] = clients['Возраст'].astype('int')
clients['Зарплата'] = clients['Зарплата'].astype('int')

### Выводы по п.1. Загрузка, обзор, предобработка данных:
1. Пропусков, выбросов нет, данных достаточно для работы. От дубликатов избавляться не стала, так как это могут быть совпадающие признаки разных клиентов. 
2. Заменила типы данных в столбцах `Возраст` и `Зарплата` на более подходящий int.
3. В таблице соблюден почти идеальный баланс между мужчинами и женщинами. Средний возраст застрахованного составляет около 31 года, при этом медиана составляет 30. Границы возраста застрахованных приводят к выводу, что компания не занимается страхованием лиц, чей возраст составляет менее 18 и старше 65 лет. Также интересно взглянуть на уровень заработной платы застрахованных - медиана составляет чуть более 40 000 рублей. В целом, в компании скорее представлен средний класс, чем слишком бедные или слишком богатые граждане. 

## 2. Проверка изменения качества линейной регрессии

In [6]:
#выделяю признаки и целевой признак:
features = clients.drop('Страховые выплаты',axis=1)
target = clients['Страховые выплаты']

features.shape, target.shape

((5000, 4), (5000,))

In [7]:
#записываю формулу линейной регрессии с единичным столбцом:
X = np.concatenate((np.ones((features.shape[0], 1)), features), axis=1) 
y = target
w = np.linalg.inv(X.T @ X) @ X.T @ y          #вычисляю параметр w по формуле минимизации MSE
print('            параметр w:', w[1:])

model = LinearRegression()                    #назначаю модель
model.fit(features, target)                   #обучаю модель
print('оценочные коэффициенты:', model.coef_) #вывожу оценочные коэффициенты для линейной регрессии

            параметр w: [ 7.92580563e-03  3.57083050e-02 -1.70081903e-07 -1.35676627e-02]
оценочные коэффициенты: [ 7.92580563e-03  3.57083050e-02 -1.70081903e-07 -1.35676627e-02]


Промежуточный вывод: параметр w и оценочные коэффициенты полностью совпадают, следовательно, качество линейной регрессии не изменяется.

In [8]:
print('  r2 для исходных признаков:', r2_score(y, model.predict(features))) #вычисляю r2 для модели

random = np.random.normal(size=(4,4)) #генерирую случайную матрицу
np.linalg.inv(random)                 #проверяю матрицу на обратимость
features_inv = features @ random      #изменяю матрицу признаков 
lr = LinearRegression()               #назначаю модель
lr.fit(features_inv, target)          #обучаю модель
print('r2 для измененных признаков:', r2_score(target, lr.predict(features_inv))) #вычисляю r2 для измененных признаков

  r2 для исходных признаков: 0.42494550308169177
r2 для измененных признаков: 0.42494550308175316


Промежуточный вывод: r2 совпадают с точностью до первых 13 знаков после запятой; 0<r2<1, значит, линейная регрессия работает

### Выводы по п.2. Проверка изменения качества линейной регрессии:

Параметр w и оценочные коэффициенты, оценки r2 в исходной задаче и в преобразованной совпадают, следовательно, качество линейной регрессии не изменяется.

## 3. Алгоритм преобразования данных для решения задачи

Поскольку выше я изменяла признаки с помощью случайно матрицы, алгоритм преобразования данных для защиты информации предлагаю следующий:
1. Генерирую случайную квадратную матрицу random
2. Умножаю матрицу признаков на случайную квадратную матрицу random и на константу PI = 3.14

In [9]:
PI = 3.14

random = np.random.normal(size=(4,4))   #генерирую случайную матрицу
np.linalg.inv(random)                   #проверяю матрицу на обратимость
features_inv = features @ random * PI   #умножаю матрицу признаков на случайную  матрицу и на PI 
features_inv.head(5)

,0,1,2,3
0,-108850.445963,-25385.633470,131021.059677,-13129.172191
1,-83404.808526,-19420.946564,100404.581204,-10022.242012
2,-46093.579358,-10722.891582,55493.378847,-5531.623873
3,-91508.133836,-21378.278118,110131.836200,-11065.012204
4,-57280.559158,-13340.892814,68954.276977,-6895.904160


In [10]:
lr = LinearRegression()               #назначаю модель
lr.fit(features_inv, target)          #обучаю модель
print('r2:',
      r2_score(target,
               lr.predict(features_inv))) #вычисляю r2 для измененных признаков

r2: 0.4249455030816913


### Выводы по п.3. Алгоритм преобразования данных для решения задачи:
r2 на защищенных данных совпадает с r2 исходных данных, следовательно, алгоритм справляется с защитой данных

In [11]:
print('Исходные признаки:')
display(features.head(5)) #вывожу посмотреть исходные признаки

print()

decode = round(features_inv @ np.linalg.inv(random) * (1/PI)).astype(int)  #можно и без округления и приведения к целочисленному типу
decode.columns = ['Пол', 'Возраст', 'Зарплата', 'Члены семьи'] #можно и без замены названия признаков, так как это снижает защиту данных
print('Декодированные признаки:')
display(decode.head(5)) #вывожу посмотреть декодированные признаки

Исходные признаки:


,Пол,Возраст,Зарплата,Члены семьи
0,1,41,49600,1
1,0,46,38000,1
2,0,29,21000,0
3,0,21,41700,2
4,1,28,26100,0



Декодированные признаки:


,Пол,Возраст,Зарплата,Члены семьи
0,1,41,49600,1
1,0,46,38000,1
2,0,29,21000,0
3,0,21,41700,2
4,1,28,26100,0
